In [1]:
import numpy as np
from sklearn.metrics import f1_score

class Layer:
    def __init__(self, input_size=None, output_size=None, weights=None, bias=None):
        self.input = None
        self.output = None
        if (input_size is None or output_size is None) and (weights is None or bias is None):
            raise Exception('input and output sizes or weights with bias are required')
        if weights is None:
            self.weights = np.random.rand(input_size, output_size) # uniform
        else:
            self.weights = weights
            self.input_size = len(self.weights)
            self.output_size = len(self.weights[0])
        if bias is None:
            self.bias = np.random.rand(1, output_size)
        else:
            self.bias = bias

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        d_weights = -learning_rate * weights_error
        d_bias = -learning_rate * output_error
        return input_error, d_weights, d_bias
    
    def update_weights(self, d_weights, d_bias):
        self.weights += d_weights
        self.bias += d_bias
        
    
class ActivationLayer:
    def __init__(self, activation, activation_prime):
        self.input = None
        self.output = None
        self.activation = activation
        self.activation_prime = activation_prime

    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

In [2]:
from copy import deepcopy

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None

    def add(self, layer):
        self.layers.append(layer)

    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime

    def predict(self, input_data):
        samples = len(input_data)
        result = []

        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)

        return result

    def fit(self, x_train, y_train, epochs, learning_rate, batch_size):

        samples = len(x_train)
        
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        layer.update_weights(dWeight, dBias)

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
            
    def fit_momentum(self, x_train, y_train, epochs, learning_rate, batch_size, lambd):

        samples = len(x_train)
        
        momentum_w = []
        momentum_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            momentum_w.append(deepcopy(layer.weights))
            momentum_b.append(deepcopy(layer.bias))
        
        momentum_w = np.array(momentum_w, dtype=object)
        
        for i in range(len(momentum_w)):
            momentum_w[i][:] = 0
            momentum_b[i][:] = 0

        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        momentum_w[l] = dWeight + learning_rate * momentum_w[l] * lambd
                        momentum_b[l] = dBias + learning_rate * momentum_b[l] * lambd
                        layer.update_weights(momentum_w[l], momentum_b[l])

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))
            
    def fit_RMSProp(self, x_train, y_train, epochs, learning_rate, batch_size, beta):
        
        samples = len(x_train)
        
        E_w = []
        E_b = []
        
        for layer in self.layers:
            
            if isinstance(layer, ActivationLayer):
                continue  
                
            E_w.append(deepcopy(layer.weights))
            E_b.append(deepcopy(layer.bias))
        
        E_w = np.array(E_w, dtype=object)
        
        for i in range(len(E_w)):
            E_w[i][:] = 0
            E_b[i][:] = 0
            
        batch_proportion = batch_size/samples

        for i in range(epochs):
            err = 0
            
            for batch_number in range(int(1/batch_proportion)):
                
                layersDeltaWeight = []
                layersDeltaBias = []
                
                for j in range(int(batch_number * batch_size), int((batch_number+1)*batch_size)):
                    
                    #forward propagation
                    
                    output = x_train[j]
                    
                    for layer in self.layers:
                        output = layer.forward_propagation(output)

                    err += self.loss(y_train[j], output)
                    
                    #backward propagation
                    
                    error = self.loss_prime(y_train[j], output)
                    
                    tmplayerDeltaWeights = []
                    tmplayerDeltaBias = []
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, ActivationLayer):
                            
                            error = layer.backward_propagation(error, learning_rate)
                            
                        else:
                            
                            error, deltaWeights, deltaBias = layer.backward_propagation(error, learning_rate)
                            tmplayerDeltaWeights.append(deltaWeights)
                            tmplayerDeltaBias.append(deltaBias)
                        
                    layersDeltaWeight.append(tmplayerDeltaWeights)
                    layersDeltaBias.append(tmplayerDeltaBias)
                    
                for layerdWeight, layerdBias in zip(layersDeltaWeight, layersDeltaBias):
                    
                    only_full_layers = []
                    
                    for layer in reversed(self.layers):
                        
                        if isinstance(layer, Layer):
                            
                            only_full_layers.append(layer)
                            
                    l = len(only_full_layers)
                            
                    for dWeight, dBias, layer in zip(layerdWeight, layerdBias, only_full_layers):
                        
                        l -= 1
                        E_w[l] = beta * E_w[l] + (1 - beta) * dWeight ** 2
                        E_b[l] = beta * E_b[l] + (1 - beta) * dBias ** 2
                        layer.update_weights(learning_rate * dWeight / np.sqrt(E_w[l]),
                                             learning_rate * dBias / np.sqrt(E_b[l]))

            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

In [3]:
def sigmoid(x):
    e = np.exp((-1)*x)
    return 1/(1+e)

def derivative_sigmoid(x):
    e = np.exp((-1)*x)
    return e / ((1+e)**2)

def linear(x):
    return x

def derivative_linear(x):
    return np.array([1 for i in x])

In [4]:
def softmax(x):
    expX = np.exp(x)
    return expX / np.sum(expX)

def derivative_softmax(x):
    s = softmax(x)[0]
    jacobian_m = np.diag(s)
    for i in range(len(jacobian_m)):
        for j in range(len(jacobian_m[0])):
            if i == j:
                jacobian_m[i][j] = 1 - s[i] ** 2
            else:
                jacobian_m[i][j] = -1 * s[i] * s[j]
    return np.diag(jacobian_m)

In [5]:
def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size;

In [6]:
def cost_function(y_true, y_pred):  # cross-entropy
    
    y = [[0 for i in range(len(net.layers[len(net.layers)-2].output[0]))]]
    y[0][y_true[0][0]] = 1
    y = np.array(y)

    return -np.mean(y * np.log(y_pred + 1e-8))

def cost_function_prime(y_true, y_pred):
    
    y = [[0 for i in range(len(net.layers[len(net.layers)-2].output[0]))]]
    y[0][y_true[0][0]] = 1
    y = np.array(y)
    
    return y_pred - y

# easy

In [7]:
import pandas as pd
data = pd.read_csv('mio1/classification/easy-training.csv')

In [9]:
y_train = list(data.c.astype(int))

In [9]:
y_train = np.array([[[y]] for y in y_train])

In [10]:
x_train = data[['x', 'y']].values.tolist()
x_train = np.array([[x] for x in x_train])

In [11]:
np.random.seed(2203) # 2203
net = Network()
net.add(Layer(2, 100))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(100, 2))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.0001, batch_size=50)

epoch 1/1000   error=0.486116
epoch 2/1000   error=0.383070
epoch 3/1000   error=0.319551
epoch 4/1000   error=0.280396
epoch 5/1000   error=0.254347
epoch 6/1000   error=0.233263
epoch 7/1000   error=0.211419
epoch 8/1000   error=0.194005
epoch 9/1000   error=0.182800
epoch 10/1000   error=0.173635
epoch 11/1000   error=0.166719
epoch 12/1000   error=0.161461
epoch 13/1000   error=0.157034
epoch 14/1000   error=0.153094
epoch 15/1000   error=0.149491
epoch 16/1000   error=0.146135
epoch 17/1000   error=0.142967
epoch 18/1000   error=0.139952
epoch 19/1000   error=0.137074
epoch 20/1000   error=0.134326
epoch 21/1000   error=0.131700
epoch 22/1000   error=0.129185
epoch 23/1000   error=0.126769
epoch 24/1000   error=0.124474
epoch 25/1000   error=0.122358
epoch 26/1000   error=0.120459
epoch 27/1000   error=0.118740
epoch 28/1000   error=0.117129
epoch 29/1000   error=0.115567
epoch 30/1000   error=0.114021
epoch 31/1000   error=0.112474
epoch 32/1000   error=0.110918
epoch 33/1000   e

epoch 263/1000   error=0.027768
epoch 264/1000   error=0.027695
epoch 265/1000   error=0.027622
epoch 266/1000   error=0.027549
epoch 267/1000   error=0.027477
epoch 268/1000   error=0.027405
epoch 269/1000   error=0.027333
epoch 270/1000   error=0.027262
epoch 271/1000   error=0.027191
epoch 272/1000   error=0.027121
epoch 273/1000   error=0.027051
epoch 274/1000   error=0.026981
epoch 275/1000   error=0.026912
epoch 276/1000   error=0.026843
epoch 277/1000   error=0.026774
epoch 278/1000   error=0.026706
epoch 279/1000   error=0.026638
epoch 280/1000   error=0.026570
epoch 281/1000   error=0.026503
epoch 282/1000   error=0.026436
epoch 283/1000   error=0.026370
epoch 284/1000   error=0.026304
epoch 285/1000   error=0.026238
epoch 286/1000   error=0.026173
epoch 287/1000   error=0.026108
epoch 288/1000   error=0.026043
epoch 289/1000   error=0.025978
epoch 290/1000   error=0.025914
epoch 291/1000   error=0.025851
epoch 292/1000   error=0.025787
epoch 293/1000   error=0.025724
epoch 29

epoch 520/1000   error=0.016622
epoch 521/1000   error=0.016596
epoch 522/1000   error=0.016570
epoch 523/1000   error=0.016544
epoch 524/1000   error=0.016519
epoch 525/1000   error=0.016493
epoch 526/1000   error=0.016467
epoch 527/1000   error=0.016442
epoch 528/1000   error=0.016416
epoch 529/1000   error=0.016391
epoch 530/1000   error=0.016365
epoch 531/1000   error=0.016340
epoch 532/1000   error=0.016315
epoch 533/1000   error=0.016290
epoch 534/1000   error=0.016264
epoch 535/1000   error=0.016239
epoch 536/1000   error=0.016214
epoch 537/1000   error=0.016189
epoch 538/1000   error=0.016164
epoch 539/1000   error=0.016139
epoch 540/1000   error=0.016114
epoch 541/1000   error=0.016089
epoch 542/1000   error=0.016065
epoch 543/1000   error=0.016040
epoch 544/1000   error=0.016015
epoch 545/1000   error=0.015991
epoch 546/1000   error=0.015966
epoch 547/1000   error=0.015942
epoch 548/1000   error=0.015917
epoch 549/1000   error=0.015893
epoch 550/1000   error=0.015869
epoch 55

epoch 779/1000   error=0.011722
epoch 780/1000   error=0.011704
epoch 781/1000   error=0.011686
epoch 782/1000   error=0.011667
epoch 783/1000   error=0.011649
epoch 784/1000   error=0.011630
epoch 785/1000   error=0.011612
epoch 786/1000   error=0.011593
epoch 787/1000   error=0.011574
epoch 788/1000   error=0.011555
epoch 789/1000   error=0.011536
epoch 790/1000   error=0.011517
epoch 791/1000   error=0.011498
epoch 792/1000   error=0.011479
epoch 793/1000   error=0.011459
epoch 794/1000   error=0.011440
epoch 795/1000   error=0.011421
epoch 796/1000   error=0.011402
epoch 797/1000   error=0.011383
epoch 798/1000   error=0.011365
epoch 799/1000   error=0.011346
epoch 800/1000   error=0.011328
epoch 801/1000   error=0.011309
epoch 802/1000   error=0.011291
epoch 803/1000   error=0.011273
epoch 804/1000   error=0.011255
epoch 805/1000   error=0.011238
epoch 806/1000   error=0.011221
epoch 807/1000   error=0.011204
epoch 808/1000   error=0.011187
epoch 809/1000   error=0.011171
epoch 81

In [12]:
data = pd.read_csv('mio1/classification/easy-test.csv')
x_test = data[['x', 'y']].values.tolist()
y_test = list(data.c.astype(int))
x_test = np.array([[x] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)

In [13]:
y_test = list(data.c.astype(int))
out = [np.argmax(l) for l in out]
f1_score(y_test, [0 if x==0 else 1 for x in out])

0.9980119284294234

# rings3-regular

In [14]:
data = pd.read_csv('mio1/classification/rings3-regular-training.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [15]:
x_train = data[['x', 'y']].values.tolist()
x_train = np.array([[x] for x in x_train])
y_train = list(data.c)
y_train = np.array([[[y]] for y in y_train])

In [16]:
np.random.seed(2203)
net = Network()
net.add(Layer(2, 40))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(40, 20))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(20, 3))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=4500, learning_rate=0.00009, batch_size=125)

epoch 1/4500   error=0.502831
epoch 2/4500   error=0.424160
epoch 3/4500   error=0.382581
epoch 4/4500   error=0.367161
epoch 5/4500   error=0.362748
epoch 6/4500   error=0.361617
epoch 7/4500   error=0.361316
epoch 8/4500   error=0.361210
epoch 9/4500   error=0.361147
epoch 10/4500   error=0.361094
epoch 11/4500   error=0.361043
epoch 12/4500   error=0.360991
epoch 13/4500   error=0.360940
epoch 14/4500   error=0.360889
epoch 15/4500   error=0.360837
epoch 16/4500   error=0.360786
epoch 17/4500   error=0.360735
epoch 18/4500   error=0.360684
epoch 19/4500   error=0.360633
epoch 20/4500   error=0.360582
epoch 21/4500   error=0.360531
epoch 22/4500   error=0.360480
epoch 23/4500   error=0.360429
epoch 24/4500   error=0.360378
epoch 25/4500   error=0.360328
epoch 26/4500   error=0.360277
epoch 27/4500   error=0.360226
epoch 28/4500   error=0.360176
epoch 29/4500   error=0.360126
epoch 30/4500   error=0.360075
epoch 31/4500   error=0.360025
epoch 32/4500   error=0.359975
epoch 33/4500   e

epoch 261/4500   error=0.352298
epoch 262/4500   error=0.352278
epoch 263/4500   error=0.352258
epoch 264/4500   error=0.352238
epoch 265/4500   error=0.352218
epoch 266/4500   error=0.352198
epoch 267/4500   error=0.352178
epoch 268/4500   error=0.352158
epoch 269/4500   error=0.352139
epoch 270/4500   error=0.352119
epoch 271/4500   error=0.352100
epoch 272/4500   error=0.352080
epoch 273/4500   error=0.352061
epoch 274/4500   error=0.352042
epoch 275/4500   error=0.352023
epoch 276/4500   error=0.352004
epoch 277/4500   error=0.351985
epoch 278/4500   error=0.351966
epoch 279/4500   error=0.351947
epoch 280/4500   error=0.351929
epoch 281/4500   error=0.351910
epoch 282/4500   error=0.351892
epoch 283/4500   error=0.351874
epoch 284/4500   error=0.351855
epoch 285/4500   error=0.351837
epoch 286/4500   error=0.351820
epoch 287/4500   error=0.351802
epoch 288/4500   error=0.351784
epoch 289/4500   error=0.351767
epoch 290/4500   error=0.351749
epoch 291/4500   error=0.351732
epoch 29

epoch 518/4500   error=0.348202
epoch 519/4500   error=0.348189
epoch 520/4500   error=0.348176
epoch 521/4500   error=0.348163
epoch 522/4500   error=0.348150
epoch 523/4500   error=0.348137
epoch 524/4500   error=0.348124
epoch 525/4500   error=0.348111
epoch 526/4500   error=0.348098
epoch 527/4500   error=0.348086
epoch 528/4500   error=0.348073
epoch 529/4500   error=0.348060
epoch 530/4500   error=0.348047
epoch 531/4500   error=0.348035
epoch 532/4500   error=0.348022
epoch 533/4500   error=0.348009
epoch 534/4500   error=0.347997
epoch 535/4500   error=0.347984
epoch 536/4500   error=0.347972
epoch 537/4500   error=0.347959
epoch 538/4500   error=0.347946
epoch 539/4500   error=0.347934
epoch 540/4500   error=0.347921
epoch 541/4500   error=0.347909
epoch 542/4500   error=0.347896
epoch 543/4500   error=0.347884
epoch 544/4500   error=0.347871
epoch 545/4500   error=0.347858
epoch 546/4500   error=0.347846
epoch 547/4500   error=0.347833
epoch 548/4500   error=0.347821
epoch 54

epoch 775/4500   error=0.344259
epoch 776/4500   error=0.344238
epoch 777/4500   error=0.344216
epoch 778/4500   error=0.344194
epoch 779/4500   error=0.344173
epoch 780/4500   error=0.344151
epoch 781/4500   error=0.344128
epoch 782/4500   error=0.344106
epoch 783/4500   error=0.344084
epoch 784/4500   error=0.344061
epoch 785/4500   error=0.344038
epoch 786/4500   error=0.344015
epoch 787/4500   error=0.343992
epoch 788/4500   error=0.343969
epoch 789/4500   error=0.343946
epoch 790/4500   error=0.343922
epoch 791/4500   error=0.343898
epoch 792/4500   error=0.343875
epoch 793/4500   error=0.343851
epoch 794/4500   error=0.343827
epoch 795/4500   error=0.343802
epoch 796/4500   error=0.343778
epoch 797/4500   error=0.343754
epoch 798/4500   error=0.343729
epoch 799/4500   error=0.343705
epoch 800/4500   error=0.343680
epoch 801/4500   error=0.343656
epoch 802/4500   error=0.343631
epoch 803/4500   error=0.343607
epoch 804/4500   error=0.343582
epoch 805/4500   error=0.343558
epoch 80

epoch 1031/4500   error=0.336677
epoch 1032/4500   error=0.336649
epoch 1033/4500   error=0.336621
epoch 1034/4500   error=0.336592
epoch 1035/4500   error=0.336564
epoch 1036/4500   error=0.336535
epoch 1037/4500   error=0.336506
epoch 1038/4500   error=0.336477
epoch 1039/4500   error=0.336448
epoch 1040/4500   error=0.336419
epoch 1041/4500   error=0.336390
epoch 1042/4500   error=0.336360
epoch 1043/4500   error=0.336330
epoch 1044/4500   error=0.336301
epoch 1045/4500   error=0.336271
epoch 1046/4500   error=0.336240
epoch 1047/4500   error=0.336210
epoch 1048/4500   error=0.336179
epoch 1049/4500   error=0.336148
epoch 1050/4500   error=0.336117
epoch 1051/4500   error=0.336086
epoch 1052/4500   error=0.336055
epoch 1053/4500   error=0.336023
epoch 1054/4500   error=0.335992
epoch 1055/4500   error=0.335960
epoch 1056/4500   error=0.335928
epoch 1057/4500   error=0.335895
epoch 1058/4500   error=0.335863
epoch 1059/4500   error=0.335830
epoch 1060/4500   error=0.335798
epoch 1061

...

epoch 3272/4500   error=0.282636
epoch 3273/4500   error=0.282639
epoch 3274/4500   error=0.282642
epoch 3275/4500   error=0.282645
epoch 3276/4500   error=0.282648
epoch 3277/4500   error=0.282651
epoch 3278/4500   error=0.282654
epoch 3279/4500   error=0.282657
epoch 3280/4500   error=0.282661
epoch 3281/4500   error=0.282664
epoch 3282/4500   error=0.282668
epoch 3283/4500   error=0.282671
epoch 3284/4500   error=0.282675
epoch 3285/4500   error=0.282679
epoch 3286/4500   error=0.282683
epoch 3287/4500   error=0.282687
epoch 3288/4500   error=0.282691
epoch 3289/4500   error=0.282695
epoch 3290/4500   error=0.282699
epoch 3291/4500   error=0.282704
epoch 3292/4500   error=0.282708
epoch 3293/4500   error=0.282713
epoch 3294/4500   error=0.282718
epoch 3295/4500   error=0.282722
epoch 3296/4500   error=0.282727
epoch 3297/4500   error=0.282732
epoch 3298/4500   error=0.282738
epoch 3299/4500   error=0.282743
epoch 3300/4500   error=0.282749
epoch 3301/4500   error=0.282754
epoch 3302

epoch 3521/4500   error=0.279943
epoch 3522/4500   error=0.279927
epoch 3523/4500   error=0.279912
epoch 3524/4500   error=0.279897
epoch 3525/4500   error=0.279882
epoch 3526/4500   error=0.279867
epoch 3527/4500   error=0.279852
epoch 3528/4500   error=0.279836
epoch 3529/4500   error=0.279821
epoch 3530/4500   error=0.279806
epoch 3531/4500   error=0.279791
epoch 3532/4500   error=0.279776
epoch 3533/4500   error=0.279760
epoch 3534/4500   error=0.279745
epoch 3535/4500   error=0.279730
epoch 3536/4500   error=0.279715
epoch 3537/4500   error=0.279699
epoch 3538/4500   error=0.279684
epoch 3539/4500   error=0.279669
epoch 3540/4500   error=0.279653
epoch 3541/4500   error=0.279638
epoch 3542/4500   error=0.279622
epoch 3543/4500   error=0.279607
epoch 3544/4500   error=0.279591
epoch 3545/4500   error=0.279576
epoch 3546/4500   error=0.279560
epoch 3547/4500   error=0.279544
epoch 3548/4500   error=0.279529
epoch 3549/4500   error=0.279513
epoch 3550/4500   error=0.279497
epoch 3551

epoch 3770/4500   error=0.278815
epoch 3771/4500   error=0.278292
epoch 3772/4500   error=0.278779
epoch 3773/4500   error=0.278067
epoch 3774/4500   error=0.278525
epoch 3775/4500   error=0.278580
epoch 3776/4500   error=0.278146
epoch 3777/4500   error=0.278587
epoch 3778/4500   error=0.277937
epoch 3779/4500   error=0.278157
epoch 3780/4500   error=0.278567
epoch 3781/4500   error=0.277413
epoch 3782/4500   error=0.277319
epoch 3783/4500   error=0.277720
epoch 3784/4500   error=0.277360
epoch 3785/4500   error=0.277356
epoch 3786/4500   error=0.276860
epoch 3787/4500   error=0.277101
epoch 3788/4500   error=0.277984
epoch 3789/4500   error=0.278808
epoch 3790/4500   error=0.279069
epoch 3791/4500   error=0.279242
epoch 3792/4500   error=0.279384
epoch 3793/4500   error=0.279469
epoch 3794/4500   error=0.279507
epoch 3795/4500   error=0.279516
epoch 3796/4500   error=0.279509
epoch 3797/4500   error=0.279493
epoch 3798/4500   error=0.279472
epoch 3799/4500   error=0.279449
epoch 3800

epoch 4019/4500   error=0.275718
epoch 4020/4500   error=0.275706
epoch 4021/4500   error=0.275694
epoch 4022/4500   error=0.275681
epoch 4023/4500   error=0.275669
epoch 4024/4500   error=0.275657
epoch 4025/4500   error=0.275645
epoch 4026/4500   error=0.275632
epoch 4027/4500   error=0.275620
epoch 4028/4500   error=0.275608
epoch 4029/4500   error=0.275596
epoch 4030/4500   error=0.275585
epoch 4031/4500   error=0.275573
epoch 4032/4500   error=0.275561
epoch 4033/4500   error=0.275549
epoch 4034/4500   error=0.275538
epoch 4035/4500   error=0.275526
epoch 4036/4500   error=0.275515
epoch 4037/4500   error=0.275503
epoch 4038/4500   error=0.275492
epoch 4039/4500   error=0.275481
epoch 4040/4500   error=0.275469
epoch 4041/4500   error=0.275458
epoch 4042/4500   error=0.275447
epoch 4043/4500   error=0.275436
epoch 4044/4500   error=0.275425
epoch 4045/4500   error=0.275414
epoch 4046/4500   error=0.275403
epoch 4047/4500   error=0.275393
epoch 4048/4500   error=0.275382
epoch 4049

epoch 4268/4500   error=0.274717
epoch 4269/4500   error=0.274712
epoch 4270/4500   error=0.274707
epoch 4271/4500   error=0.274702
epoch 4272/4500   error=0.274697
epoch 4273/4500   error=0.274692
epoch 4274/4500   error=0.274688
epoch 4275/4500   error=0.274683
epoch 4276/4500   error=0.274678
epoch 4277/4500   error=0.274673
epoch 4278/4500   error=0.274668
epoch 4279/4500   error=0.274663
epoch 4280/4500   error=0.274658
epoch 4281/4500   error=0.274653
epoch 4282/4500   error=0.274648
epoch 4283/4500   error=0.274644
epoch 4284/4500   error=0.274639
epoch 4285/4500   error=0.274634
epoch 4286/4500   error=0.274629
epoch 4287/4500   error=0.274624
epoch 4288/4500   error=0.274619
epoch 4289/4500   error=0.274614
epoch 4290/4500   error=0.274609
epoch 4291/4500   error=0.274605
epoch 4292/4500   error=0.274600
epoch 4293/4500   error=0.274595
epoch 4294/4500   error=0.274590
epoch 4295/4500   error=0.274585
epoch 4296/4500   error=0.274580
epoch 4297/4500   error=0.274576
epoch 4298

In [17]:
data = pd.read_csv('mio1/classification/rings3-regular-test.csv')
x_test = data[['x', 'y']].values.tolist()
y_test = list(data.c)
x_test = np.array([[x] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)

In [18]:
y_test = list(data.c)
out = [np.argmax(l) for l in out]
f1_score(y_test, out, average = 'weighted')

0.6880433315409455

# xor3

In [19]:
data = pd.read_csv('mio1/classification/xor3-training.csv')
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

In [20]:
x_train = data[['x', 'y']].values.tolist()
x_train = np.array([[x] for x in x_train])
y_train = list(data.c.astype(int))
y_train = np.array([[[y]] for y in y_train])

In [22]:
np.random.seed(42) # 2203
net = Network()
net.add(Layer(2, 9))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(9, 9))
net.add(ActivationLayer(sigmoid, derivative_sigmoid))
net.add(Layer(9, 2))
net.add(ActivationLayer(softmax, derivative_softmax))

net.use(cost_function, cost_function_prime)
net.fit(x_train, y_train, epochs=500, learning_rate=0.005, batch_size=100)

epoch 1/500   error=0.360273
epoch 2/500   error=0.358991
epoch 3/500   error=0.358691
epoch 4/500   error=0.358403
epoch 5/500   error=0.358068
epoch 6/500   error=0.357622
epoch 7/500   error=0.357137
epoch 8/500   error=0.356568
epoch 9/500   error=0.356038
epoch 10/500   error=0.355478
epoch 11/500   error=0.354920
epoch 12/500   error=0.354368
epoch 13/500   error=0.353800
epoch 14/500   error=0.353201
epoch 15/500   error=0.352562
epoch 16/500   error=0.351917
epoch 17/500   error=0.351281
epoch 18/500   error=0.350647
epoch 19/500   error=0.350006
epoch 20/500   error=0.349341
epoch 21/500   error=0.348648
epoch 22/500   error=0.347896
epoch 23/500   error=0.347017
epoch 24/500   error=0.345751
epoch 25/500   error=0.343852
epoch 26/500   error=0.341801
epoch 27/500   error=0.340390
epoch 28/500   error=0.339329
epoch 29/500   error=0.338268
epoch 30/500   error=0.337207
epoch 31/500   error=0.336151
epoch 32/500   error=0.335107
epoch 33/500   error=0.334083
epoch 34/500   erro

epoch 270/500   error=0.312091
epoch 271/500   error=0.312453
epoch 272/500   error=0.313251
epoch 273/500   error=0.312284
epoch 274/500   error=0.312534
epoch 275/500   error=0.311964
epoch 276/500   error=0.313407
epoch 277/500   error=0.313315
epoch 278/500   error=0.312302
epoch 279/500   error=0.312311
epoch 280/500   error=0.312315
epoch 281/500   error=0.312539
epoch 282/500   error=0.312311
epoch 283/500   error=0.312473
epoch 284/500   error=0.312032
epoch 285/500   error=0.312213
epoch 286/500   error=0.312395
epoch 287/500   error=0.312180
epoch 288/500   error=0.312863
epoch 289/500   error=0.312826
epoch 290/500   error=0.312848
epoch 291/500   error=0.312373
epoch 292/500   error=0.312908
epoch 293/500   error=0.312871
epoch 294/500   error=0.312496
epoch 295/500   error=0.311840
epoch 296/500   error=0.312383
epoch 297/500   error=0.312177
epoch 298/500   error=0.312557
epoch 299/500   error=0.312944
epoch 300/500   error=0.313170
epoch 301/500   error=0.312517
epoch 30

/var/folders/9m/_mlrk6qn06jb51wp2v289q1c0000gn/T/ipykernel_3137/3504725208.py:7: RuntimeWarning: overflow encountered in square
  return e / ((1+e)**2)


epoch 458/500   error=0.315751
epoch 459/500   error=0.315532
epoch 460/500   error=0.314672
epoch 461/500   error=0.315060
epoch 462/500   error=0.315949
epoch 463/500   error=0.315830
epoch 464/500   error=0.315270
epoch 465/500   error=0.315972
epoch 466/500   error=0.315410
epoch 467/500   error=0.315088
epoch 468/500   error=0.315959
epoch 469/500   error=0.315798
epoch 470/500   error=0.315214
epoch 471/500   error=0.316202
epoch 472/500   error=0.316109
epoch 473/500   error=0.315675
epoch 474/500   error=0.316930
epoch 475/500   error=0.316098
epoch 476/500   error=0.315240
epoch 477/500   error=0.315552
epoch 478/500   error=0.315631
epoch 479/500   error=0.316383
epoch 480/500   error=0.316483
epoch 481/500   error=0.315524
epoch 482/500   error=0.316788
epoch 483/500   error=0.316457
epoch 484/500   error=0.315798
epoch 485/500   error=0.316936
epoch 486/500   error=0.316842
epoch 487/500   error=0.316329
epoch 488/500   error=0.316387
epoch 489/500   error=0.317149
epoch 49

In [23]:
data = pd.read_csv('mio1/classification/xor3-test.csv')
x_test = data[['x', 'y']].values.tolist()
y_test = list(data.c.astype(int))
x_test = np.array([[x] for x in x_test])
y_test = np.array([[[y]] for y in y_test])
out = net.predict(x_test)

In [24]:
y_test = list(data.c.astype(int))
out = [np.argmax(l) for l in out]
f1_score(y_test, out)

0.6104294478527608